In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
import matplotlib.pyplot as plt
from evallm.experiments.sequence_completion_summary import (
    sequence_completion_results,
    display_prompt as display_prompt_sc,
    sequence_completion_null_results,
)

from evallm.experiments.transducer_summary import (
    transducer_results,
    transducer_null_results,
)
from evallm.experiments.main_tables import (
    best_prompt,
    multi_prompts,
    main_table_of_results,
    multi_prompt_table_of_results,
    plot_transducer_vs_sequence_completion,
    best_prompt,
    all_p_values,
)
from evallm.experiments.main_tables import plot_significance

from evallm.experiments.transducer_plotting import produce_table
from evallm.experiments.models_display import model_by_display_key

In [5]:
null_t = transducer_null_results()
null_sc = sequence_completion_null_results()
results_t = transducer_results()
results_sc = sequence_completion_results()

### Main Table

In [6]:
main_table_of_results(best_prompt(results_t), best_prompt(results_sc))

{\renewcommand{\arraystretch}{1.25}\begin{tabular}{l|ccc|ccccc}
\hline
\bf Model & \bf Size & \bf IT? & \bf Code? & \bf Sequence Completion & \bf SR & \bf Transducer & \bf TR\\
\hline
\multicolumn{8}{c}{ \bf Baselines} \\
\hline
\textsc{BruteForce} & -- &  &  & \cellcolor{lightgray} 100.0 (99.9--100.0) & 1 & \cellcolor{lightgray} 96.4 (96.2--96.7) & 1 \\
\hline
6-\textsc{Gram} & -- &  &  & \bf 91.7 (91.0--92.4) & 2 & \bf 93.5 (93.1--93.9) & 2 \\
\hline
5-\textsc{Gram} & -- &  &  & 91.2 (90.4--91.9) & 3 & 93.4 (93.0--93.7) & 3 \\
\hline
4-\textsc{Gram} & -- &  &  & 89.6 (88.7--90.4) & 4 & 91.1 (90.6--91.6) & 5 \\
\hline
3-\textsc{Gram} & -- &  &  & 87.0 (86.1--87.8) & 5 & 87.0 (86.4--87.6) & 21 \\
\hline
2-\textsc{Gram} & -- &  &  & 83.3 (82.2--84.2) & 8 & 74.5 (73.6--75.3) & 29 \\
\hline
\textsc{Common-Suffix} & -- &  &  & 84.7 (83.6--85.6) & 6 & -- & -- \\
\hline
\textsc{Random}$_S$/\textsc{Null}$_T$ & -- &  &  & 53.3 (51.7--54.7) & 31 & 68.9 (68.2--69.6) & 30 \\
\hline
\multicolumn{8

### Comparison of Prompts

In [7]:
multi_prompt_table_of_results(multi_prompts(results_t), multi_prompts(results_sc))

{\renewcommand{\arraystretch}{1.25}\begin{tabular}{l|ccccc}
\hline
\bf Model & \bf \textsc{Basic} & \bf \textsc{Basic-COT} & \bf \textsc{More-Expl} & \bf \textsc{More-Expl-COT} & \bf \textsc{Red-Green} \\
\hline
\multicolumn{6}{l}{ \bf Sequence Completion} \\
\hline
gpt-4o-mini & \bf 72.4 (68.1--76.3) & 60.0 (55.8--64.4) & 70.5 (66.4--74.6) & 58.0 (53.4--62.4) & 59.1 (54.9--63.2) \\
\hline
gpt-4o & 72.1 (65.9--78.2) & \bf 74.8 (69.3--80.4) & N/A & 67.4 (60.8--73.8) & 74.4 (69.9--78.6) \\
\hline
claude-3.5 & N/A & 82.8 (77.5--87.5) & N/A & \bf 84.0 (79.3--88.4) & 80.0 (74.9--85.2) \\
\hline
o3-mini & N/A & \bf 81.1 (76.0--85.8) & N/A & 58.2 (49.6--66.8) & 69.8 (64.4--75.0) \\
\hline
\multicolumn{6}{l}{ \bf Transducer} \\
\hline
gpt-4o-mini & \bf 79.8 (77.3--82.2) & 66.5 (64.3--68.7) & 76.7 (74.2--79.3) & 65.2 (63.1--67.4) & 74.5 (72.0--77.0) \\
\hline
gpt-4o & \bf 83.7 (80.1--86.9) & 67.8 (62.4--73.2) & 82.6 (79.1--85.9) & 67.8 (63.1--72.3) & 82.6 (78.8--86.3) \\
\hline
claude-3.5 & 86.

### Commas

In [ ]:
produce_table(
    {k: v for k, v in results_sc.items() if k in model_by_display_key},
    [display_prompt_sc(p) for p in ["Basic", "Basic-Commas"]],
)

In [ ]:
plt.figure(figsize=(8, 4), dpi=400, facecolor="white", tight_layout=True)
plot_transducer_vs_sequence_completion(results_sc, results_t)
plt.savefig("../output/sequence_completion_vs_transducer.png")

### Null

In [ ]:
print(r"{\scriptsize")
multi_prompt_table_of_results(
    multi_prompts(null_t, minimum_number_prompts=1),
    multi_prompts(null_sc, minimum_number_prompts=1),
    bold_best=False,
)
print("}")

In [ ]:
from evallm.experiments.main_tables import flat_pandas_table, reorderings

In [ ]:
import numpy as np


def flat(result):
    return {(k1, k2): np.mean(v) for k1, k2v in result.items() for k2, v in k2v.items()}


for k, v in flat(null_sc).items():
    if v < 0.05:
        continue
    if "Commas" in k[1]:
        continue
    print(k, v)
    # print(k, np.mean(v))

all_vals = np.array(
    [
        np.mean(v)
        for null in (null_t, null_sc)
        for vs in null.values()
        for prompt, v in vs.items()
        if "Commas" not in prompt
    ]
)

all_vals[all_vals < 0.5].max(), all_vals[all_vals >= 0.6].min()

In [ ]:
table_sc = flat_pandas_table(results_sc, null_sc)
table_t = flat_pandas_table(results_t, null_t)

In [ ]:
table_sc

In [ ]:
print("Sequence just same model")
reorderings(table_sc, same_model=True)
print()
print("Transducer just same model")
reorderings(table_t, same_model=True)

In [ ]:
print("Sequence just best prompt")
reorderings(table_sc, only_best_prompt=True)
print()
print("Transducer just best prompt")
reorderings(table_t, only_best_prompt=True)

In [ ]:
flat_t = best_prompt(results_t)
ps_t = all_p_values(flat_t)
flat_sc = {
    k: v
    for k, v in best_prompt(results_sc).items()
    if not isinstance(v, float) or not np.isnan(v)
}
ps_sc = all_p_values(flat_sc)

In [ ]:
size = 6
_, axs = plt.subplots(1, 2, figsize=(size * 2, size), dpi=400, tight_layout=True)
plot_significance(axs[0], flat_sc, ps_sc)
axs[0].set_title("Sequence Completion")
plot_significance(axs[1], flat_t, ps_t)
axs[1].set_title("Transducer")
plt.savefig("../output/significance.png")
plt.show()